In [1]:
import statsapi
import json


In [3]:
sched = statsapi.schedule(start_date='07/01/2018',end_date='07/31/2018')
print(sched)

[{'game_id': 530661, 'game_datetime': '2018-07-01T17:05:00Z', 'game_date': '2018-07-01', 'game_type': 'R', 'status': 'Final', 'away_name': 'Los Angeles Angels', 'home_name': 'Baltimore Orioles', 'away_id': 108, 'home_id': 110, 'doubleheader': 'N', 'game_num': 1, 'home_probable_pitcher': 'Kevin Gausman', 'away_probable_pitcher': 'Deck McGuire', 'home_pitcher_note': 'The righty hasn’t won since May 11 despite quality starts in three of his past four outings. Gausman went six strong innings against the Mariners on Tuesday and got a no-decision. He’ll look for some offensive help when he takes on the Angels.', 'away_pitcher_note': 'McGuire will make his first start for the Angels on Sunday, filling in for the injured John Lamb (shoulder/elbow). McGuire last pitched Tuesday, when he gave up three runs over four innings in a relief appearance against the Red Sox.', 'away_score': 2, 'home_score': 8, 'current_inning': 9, 'inning_state': 'Top', 'venue_id': 2, 'venue_name': 'Oriole Park at Camde

In [18]:
teams = statsapi.lookup_team("", activeStatus="Y", season=None, sportIds=1)
for team in teams:
    with open("teamLookup.json", "a") as f:
        f.write(json.dumps(team)+"\n")

    
    

In [43]:
today = statsapi.datetime.today().date()

schedule = statsapi.schedule(
    date=today,
    sportId=1
)
for match in schedule:
    with open("today_schedule.json", "a") as f:
        f.write(json.dumps(match)+"\n")

In [ ]:
#getting all matches in 2022

from datetime import date

start = date(2022,1,1)
end = date(2022,12,31)

schedule = statsapi.schedule(
    start_date=start,
    end_date=end,
)

for match in schedule:
    with open("matches2022.json", "a") as f:
        f.write(json.dumps(match)+"\n")
    
    game_id = match.get("game_id", None)
    game_linescore = statsapi.get("game_linescore",{"gamePk":game_id})
    with open("linescores2022.json", "a") as f2:
        f2.write(json.dumps(game_linescore)+"\n")


In [61]:
year = 2022

standings = statsapi.standings_data(
    leagueId="103,104",
    division="all",
    include_wildcard=True,
    season="2023"
)


with open("standings2023.json", "a") as f:
    f.write(json.dumps(standings)+"\n")

In [4]:
stat = statsapi.player_stat_data(660271, group="[hitting,pitching,fielding]", type="[career,yearByYear]", sportId=1)
with open("stat.json", "w") as f:
    f.write(json.dumps(stat)+"\n")

In [72]:
team_stat = (statsapi.get('team_stats', {'teamId':119,'season':2021,'group':'hitting,pitching','stats':'season'}))

with open("team_stat.json", "w") as f:
    f.write(json.dumps(team_stat)+"\n")

# 본격적인 데이터 추출

In [ ]:
# 연도별 팀 id 저장
for year in range(1876, 2024):
    team_ids = []
    teams = statsapi.get("teams", {"season": year, "sportId":1}).get("teams")
    for team in teams:
        team_id = team["id"]
        team_name = team["name"]
        team_ids.append({"id": team_id, "name": team_name})
    with open(f"team_ids/team_ids{year}.json", "w") as f:        
        f.write(json.dumps(team_ids))

In [113]:
# season 기반 team_standing 추출

# 1969년 이전은 division이 나눠져 있지 않아서 데이터를 따로 처리해야함.
for year in range(1969,2024):
    standings = statsapi.standings_data(
        leagueId="103,104",
        division="all",
        include_wildcard=True,
        season = year
    )
    filename = f"standing{year}.json"
    with open("teamstandings/"+filename, "a") as f:
        f.write(json.dumps(standings)+"\n")
    

In [19]:
# season별 teamId 기반 team_stat 추출

for year in range(2023, 2024):
    with open(f"team_ids/team_ids{year}.json", "r") as f:
        teams = json.load(f)
        teams_stat = []
        for team in teams:
            team_id = team["id"]
            teams_stat.append(statsapi.get('team_stats', {'teamId':team_id,'season':year,'group':'hitting,pitching,fielding','stats':'season'}))
            with open(f"teamstats/team_stat{year}.json", "w") as f:
                f.write(json.dumps(teams_stat))
# for year in range(1999, 2024):
#     for team in team_ids:
#         team_id = team.get("id")
#         team_stat = (statsapi.get('team_stats', {'teamId':team_id,'season':year,'group':'hitting,pitching,fielding','stats':'season'}))

#         with open(f"teamstats/team_stat{year}.json", "a") as f:
#             f.write(json.dumps(team_stat)+"\n")


In [28]:
# playerId 수집
players = statsapi.get("sports_players",{"sportId":1,"season":2023})
with open(f"test.json", "w") as f:
        f.write(json.dumps(players))

In [ ]:
# playerId 기반 

In [3]:
# matchId 수집
from datetime import date

for year in range(1901,2024):
    start = date(year,1,1)
    end = date(year,12,31)

    matches = statsapi.schedule(
        start_date=start,
        end_date=end,
        sportId=1
    )

    match_ids = []
    for match in matches:
        game_id = match["game_id"]
        summary = match["summary"]
        match_ids.append({"game_id":game_id,"summary":summary})
    with open(f"match_ids/match_ids{year}.json","w") as f:
        f.write(json.dumps(match_ids))


Excluding seriesStatus hydration because the MLB API throws an error for 2014-03-11 which is included in the requested date range.


In [ ]:
import ipyparallel as ipp
# request a cluster
with ipp.Cluster() as rc:
    # get a view on the cluster
    view = rc.load_balanced_view()
    # submit the tasks
    asyncresult = view.map_async(print("helloworld"))
    # wait interactively for results
    asyncresult.wait_interactive()
    # retrieve actual results
    result = asyncresult.get()
# at this point, the cluster processes have been shutdown

In [ ]:
# matchId별 linescore 수집
import json
def get_linescores(start, end):
    for year in range(start, end):
        with open(f"match_ids/match_ids{year}.json", "r") as f:
            match_ids = json.load(f)
        
        linescores = []

        for match_info in match_ids:
            game_id = match_info.get("game_id", None)
            game_linescore = statsapi.get("game_linescore",{"gamePk":game_id})
            linescores.append({"game_id":game_id,"linescore":game_linescore})
        with open(f"linescores/linescores{year}.json", "w") as f2:
            f2.write(json.dumps(linescores))

In [ ]:
# matchId별 linescore 수집
import json
import statsapi

for year in range(1916, 1920):
    with open(f"match_ids/match_ids{year}.json", "r") as f:
        match_ids = json.load(f)
    
    linescores = []

    for match_info in match_ids:
        game_id = match_info.get("game_id", None)
        game_linescore = statsapi.get("game_linescore",{"gamePk":game_id})
        linescores.append({"game_id":game_id,"linescore":game_linescore})
    with open(f"linescores/linescores{year}.json", "w") as f2:
        f2.write(json.dumps(linescores))

In [ ]:
# matchId별 linescore 수집
import json

for year in range(1930, 1940):
    with open(f"match_ids/match_ids{year}.json", "r") as f:
        match_ids = json.load(f)
    
    linescores = []

    for match_info in match_ids:
        game_id = match_info.get("game_id", None)
        game_linescore = statsapi.get("game_linescore",{"gamePk":game_id})
        linescores.append({"game_id":game_id,"linescore":game_linescore})
    with open(f"linescores/linescores{year}.json", "w") as f2:
        f2.write(json.dumps(linescores))

In [ ]:
# matchId별 linescore 수집
import json

for year in range(1940, 1950):
    with open(f"match_ids/match_ids{year}.json", "r") as f:
        match_ids = json.load(f)
    
    linescores = []

    for match_info in match_ids:
        game_id = match_info.get("game_id", None)
        game_linescore = statsapi.get("game_linescore",{"gamePk":game_id})
        linescores.append({"game_id":game_id,"linescore":game_linescore})
    with open(f"linescores/linescores{year}.json", "w") as f2:
        f2.write(json.dumps(linescores))

In [14]:
# season별 선수 ID 수집
import json

for year in range(1876, 2023):
    players_info = statsapi.get("sports_players",{"sportId":1,"season":year}).get("people")
    players_id = []
    for player_info in players_info:
        player_id = player_info.get("id",None)
        player_name = player_info.get("fullName",None)
        players_id.append({"id":player_id, "fullName":player_name})
    with open(f"players_id/players_id{year}.json", "w") as f:
        f.write(json.dumps(players_id))

In [28]:
all_players_id = set()
for year in range(2020,2023):
    with open(f"players_id/players_id{year}.json", "r") as f:
        players_id = json.load(f)

    for player in players_id:
        all_players_id.add(str(player))

In [29]:
with open("all_players.json", "w") as f:
    f.write((json.dumps(list(all_players_id))))

In [6]:
import pandas as pd
all_players_id = pd.DataFrame()
for year in range(1876,2023):
    with open(f"players_id/players_id{year}.json", "r") as f:
        players_id = json.load(f)

    players_id_df = pd.DataFrame(players_id)

    all_players_id = pd.concat([all_players_id, players_id_df])
all_players_id = all_players_id.sort_values("id")
all_players_id = all_players_id.reset_index(drop=True)
print(all_players_id)

all_players_id = all_players_id.drop_duplicates()
all_players_id = all_players_id.reset_index(drop=True)
print(all_players_id)

all_players_id.to_json("all_players.json", orient="records")

            id       fullName
0       110001     Hank Aaron
1       110001     Hank Aaron
2       110001     Hank Aaron
3       110001     Hank Aaron
4       110001     Hank Aaron
...        ...            ...
102986  687988  Darby O'Brien
102987  687988  Darby O'Brien
102988  688010    Nate Fisher
102989  689225   Beau Brieske
102990  693821    Bryce Elder

[102991 rows x 2 columns]
           id        fullName
0      110001      Hank Aaron
1      110002    Tommie Aaron
2      110003        Don Aase
3      110005  Ed Abbaticchio
4      110006      Bert Abbey
...       ...             ...
20255  687093  Vaughn Grissom
20256  687988   Darby O'Brien
20257  688010     Nate Fisher
20258  689225    Beau Brieske
20259  693821     Bryce Elder

[20260 rows x 2 columns]


In [15]:
# player__id별
import statsapi

group="[hitting,pitching,fielding]"
type="[career, season]"

players_stat = []
for id in all_players_id["id"]:
    params = {
    "personId": id,
        "hydrate": "stats(group="
        + group
        + ",type="
        + type
        + ",sportId="
        + str(1)
        + ")",
}
    player_stat = statsapi.get("person",params)
    with open(f"players_stat/players_stat_{id}.json", "w") as f:
        f.write(json.dumps(player_stat))

KeyboardInterrupt: 

In [32]:
def player_stat_data(
    personId, group, type, sportId
):
    """Returns a list of current season or career stat data for a given player."""
    params = {
        "personId": personId,
        "hydrate": "stats(group="
        + group
        + ",type="
        + type
        + ",sportId="
        + str(sportId)
        + ")",
    }
    r = statsapi.get("person", params)

    stat_groups = []

    player = r["people"][0]

    # player = {"info":r["people"][0]}

    # for s in r["people"][0].get("stats", []):
    #     for i in range(0, len(s["splits"])):
    #         stat_group = {
    #             "type": s["type"]["displayName"],
    #             "group": s["group"]["displayName"],
    #             "season": s["splits"][i].get("season"),
    #             "stats": s["splits"][i]["stat"],
    #         }
    #         stat_groups.append(stat_group)

    # player.update({"stat_info": stat_groups})

    return player

In [33]:
# player__id별
import statsapi

players_stat = []
for id in all_players_id["id"]:
    player_stat = player_stat_data(id, group="[hitting,pitching,fielding]", type="[career,yearByYear]", sportId=1)
    with open(f"players_stat/players_stat_{id}.json", "w") as f:
        f.write(json.dumps(player_stat))

KeyboardInterrupt: 